## CONNECTIE MAKEN MET DATABASE
    Let op, pandas werk alleen met sqlalchemy connection
    Hier geen issue, bij Extraction fase wel

DROP TABLE IF EXISTS DSA_fd_stats;
CREATE TABLE DSA_fd_stats(
    DIV Varchar(4)
    ,Date Varchar (10)
    ,Hometeam Varchar(100)
    ,Awayteam Varchar(100)
    ,FTHG Varchar(10)
    ,FTAG Varchar(10)
    ,FTR Varchar(10)
    ,HTHG Varchar(10)
    ,HTAG Varchar(10)
    ,HTR Varchar(10)
    ,HSh Varchar (10)
    ,ASh Varchar (10)
    ,HST Varchar (10)
    ,AST Varchar (10)
    ,HF Varchar (10)
    ,AF Varchar (10)
    ,HC Varchar (10)
    ,AC Varchar (10)
    ,HY Varchar (10)
    ,AY Varchar (10)
    ,HR Varchar (10)
    ,AR Varchar (10)
    ,B365H Varchar(8)
    ,B365D Varchar(8)
    ,B365A Varchar(8)
    ,BSH Varchar(8)
    ,BSD Varchar(8)
    ,BSA Varchar(8)
    ,BWH Varchar(8)
    ,BWD Varchar(8)
    ,BWA Varchar(8)
    ,GBH Varchar(8)
    ,GBD Varchar(8)
    ,GBA Varchar(8)
    ,IWH Varchar(8)
    ,IWD Varchar(8)
    ,IWA Varchar(8)
    ,LBH Varchar(8)
    ,LBD Varchar(8)
    ,LBA Varchar(8)
    ,PSH Varchar(8)
    ,PSD Varchar(8)
    ,PSA Varchar(8)
    ,SOH Varchar(8)
    ,SOD Varchar(8)
    ,SOA Varchar(8)
    ,SBH Varchar(8)
    ,SBD Varchar(8)
    ,SBA Varchar(8)
    ,SJH Varchar(8)
    ,SJD Varchar(8)
    ,SJA Varchar(8)
    ,SYH Varchar(8)
    ,SYD Varchar(8)
    ,SYA Varchar(8)
    ,VCH Varchar(8)
    ,VCD Varchar(8)
    ,VCA Varchar(8)
    ,WHH Varchar(8)
    ,WHD Varchar(8)
    ,WHA Varchar(8)
    );

    
DROP TABLE IF EXISTS DSA_oddapi_odds;
CREATE TABLE DSA_oddapi_odds(
    Div Varchar (100)
    ,Datum Varchar(100)
    ,Hometeam Varchar(100)
    ,Awayteam Varchar(100)
    ,Wedkantoor Varchar(100)
    ,h_odd_price Varchar(10)
    );

    
DROP TABLE IF EXISTS DSA_update;
CREATE TABLE DSA_update(
    tabelnaam Varchar (100)
    ,update_datum Date
    );


INSERT INTO DSA_update (tabelnaam, update_datum)
VALUES ('DWA_d_datum','1999-01-01')
        ,('DWA_d_teams','1999-01-01')
        ,('DWA_d_wedkantoren','1999-01-01')
        ,('DWA_d_wedstrijden','1999-01-01')
        ,('DWA_f_odds','1999-01-01')
        ,('DWA_f_resultaten','1999-01-01')
        ,('DWA_f_vorm','1999-01-01');

        
DROP VIEW IF EXISTS DSA_fd_stats_resultaten_VIEW;
CREATE VIEW DSA_fd_stats_resultaten_VIEW AS
SELECT 
div
,datum
,hometeam
,awayteam
,fthg
,ftag
,ftr
,hthg 
,htag 
,htr 
,hsh 
,ash 
,hst 
,ast
,hf 
,af 
,hc
,ac 
,hy 
,ay 
,hr 
,ar 
FROM (SELECT  
	        row_number() over (partition by hometeam, awayteam, div, date) nr
			,date 
			,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	        ,div
	        ,dt.team hometeam
	        ,dt2.team awayteam
	        ,fthg
	        ,ftag
	        ,ftr
	        ,hthg 
	        ,htag 
	        ,htr 
	        ,hsh 
	        ,ash 
	        ,hst 
	        ,ast
	        ,hf 
	        ,af 
	        ,hc
	        ,ac 
	        ,hy 
	        ,ay 
	        ,hr 
	        ,ar 
	FROM DSA_fd_stats fs2 
	LEFT OUTER JOIN DWA_d_teams dt 
		on hometeam = dt.team_fd 
	LEFT OUTER JOIN DWA_d_teams dt2 
		on awayteam = dt2.team_fd 
) bron
WHERE nr = 1
ORDER BY div, datum;


DROP VIEW DSA_oddapi_odds_VIEW;
CREATE VIEW DSA_oddapi_odds_VIEW AS
SELECT
div
,datum
,hometeam
,awayteam
,wedkantoor
,h_odd_price
FROM (
	SELECT 
	row_number() over (partition by dt.team, dt2.team, wk.wedkantoor, datum) as rownr
	,oo.div div
	,date(oo.datum) datum
	,dt.team as hometeam
	,dt2.team as awayteam
	,wk.wedkantoor as wedkantoor
	,oo.h_odd_price
	FROM DSA_oddapi_odds oo 
	LEFT OUTER JOIN DWA_d_teams dt 
		on hometeam = dt.team_oddapi  
	LEFT OUTER JOIN DWA_d_teams dt2 
		on awayteam = dt2.team_oddapi 
	LEFT OUTER JOIN DWA_d_wedkantoren wk
		on oo.wedkantoor = wk.wk_oddapi
	ORDER BY div, datum) sel
WHERE rownr > 1
;


DROP VIEW DSA_fd_stats_odds_VIEW;
CREATE VIEW DSA_fd_stats_odds_VIEW AS
select 
odds.div
,odds.datum
,dt.team as hometeam
,dt2.team as awayteam
,wk.wedkantoor as wedkantoor
,h_odd_price
from (
	select 
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'B365' wedkantoor
	,b365h as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'BS' wedkantoor
	,bsh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'BW' wedkantoor
	,bwh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'GB' wedkantoor
	,gbh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'IW' wedkantoor
	,iwh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'LB' wedkantoor
	,lbh as h_odd_price
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'PS' wedkantoor
	,psh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'SO' wedkantoor
	,soh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'SB' wedkantoor
	,sbh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'SJ' wedkantoor
	,sjh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'SY' wedkantoor
	,syh as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'VC' wedkantoor
	,vch as h_odd_price 
	from DSA_fd_stats
	union
	select
	div
	,date((2000+substr(date,-2))||'-'||substr(date,4,2)||'-'||substr(date,1,2)) datum
	,hometeam 
	,awayteam
	,'WH' wedkantoor
	,whh as h_odd_price 
	from DSA_fd_stats 
	) odds
left outer join DWA_d_teams dt 
	on odds.hometeam = dt.team_fd  
left outer join DWA_d_teams dt2 
	on odds.awayteam = dt2.team_fd
left outer join DWA_d_wedkantoren wk
	on odds.wedkantoor = wk.wk_fd 
order by div, datum
;

DROP table IF EXISTS CONTROLE_wedstrijdaantallen;
CREATE table CONTROLE_wedstrijdaantallen (
div varchar (5)
,seizoen varchar (20)
,aantal int(4)
);

DROP TABLE IF EXISTS DSA_bets;
CREATE TABLE DSA_bets(
speelronde varchar (10)
,seizoen varchar (20)
,competitie varchar (10)
,datum varchar (20)
,s_wedstrijd varchar (20)
,hometeam varchar (50)
,awayteam varchar (50)
,best_odd varchar (20)
,min_odd varchar (20)
,rea_inzet varchar (20)
,rea_odd varchar (20)
,wedkantoor varchar (50)
);

drop view DSA_bets_VIEW;
create view DSA_bets_VIEW as
	select 
	cast(speelronde as int) speelronde,
	date(datum) datum,
	s_wedstrijd,
	cast(replace(best_odd, ',', '.') as decimal(10,2)) best_odd,
	case when min_odd = 'nan' then null else cast(replace(min_odd, ',','.') as decimal(10,2)) end min_odd,
	case when rea_inzet = 'nan' then null else cast(rea_inzet as decimal(10,0)) end rea_inzet,
	cast(replace(rea_odd, ',', '.') as decimal(10,2)) rea_odd,
	case when wedkantoor = 'nan' then null else wedkantoor end wedkantoor
	from DSA_bets
order by datum, speelronde
;

